# Experimenting with Feedback Analysis using Sentiment Analysis and Collaborative Filtering

## Data

Ingredient database contains features such as type, colour and taste features

In [ ]:
import pandas as pd
from utils.process_data import get_data

# Fetch data
ingredient_df = get_data()

# Adjust pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Print the specific columns
print(ingredient_df[['Category7', 'Texture', 'Taste', 'Colour']])


### Child Feature Data

Generated data on 30 children, including features such as age, gender, favorite_cuisine, and health considerations. These features are known to influence food preferences. Such data could be collected through existing databases. These features paired with the ingredient features could be used to predict whether a new child will like an ingredient.

In [ ]:
def get_child_data():
    # Function to get feature data on children
    return {
        "child1": {"age": 10, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child2": {"age": 9, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "Italian"},
        "child3": {"age": 9, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child4": {"age": 9, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child5": {"age": 11, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child6": {"age": 11, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child7": {"age": 9, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child8": {"age": 9, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "Italian"},
        "child9": {"age": 10, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child10": {"age": 11, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child11": {"age": 9, "gender": "F", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child12": {"age": 9, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child13": {"age": 9, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Seafood"},
        "child14": {"age": 10, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Seafood"},
        "child15": {"age": 11, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Seafood"},
        "child16": {"age": 11, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Seafood"},
        "child17": {"age": 9, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child18": {"age": 9, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Seafood"},
        "child19": {"age": 9, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Seafood"},
        "child20": {"age": 10, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "BBQ"},
        "child21": {"age": 10, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "BBQ"},
        "child22": {"age": 9, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "BBQ"},
        "child23": {"age": 9, "gender": "F", "health_consideration": "moderately health conscious", "favorite_cuisine": "BBQ"},
        "child24": {"age": 9, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "BBQ"},
        "child25": {"age": 11, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "BBQ"},
        "child26": {"age": 11, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "BBQ"},
        "child27": {"age": 9, "gender": "F", "health_consideration": "moderately health conscious", "favorite_cuisine": "Seafood"},
        "child28": {"age": 9, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Seafood"},
        "child29": {"age": 9, "gender": "M", "health_consideration": "very health conscious", "favorite_cuisine": "Italian"},
        "child30": {"age": 10, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"}
    }

### Child Preference Data

The dataset consists of generated preferences for n children. In a real-world context, such data could be gathered through questionnaires or feedback after meals. For this example, the data is synthesized based on multiple studies that explore factors influencing children's food preferences. Each child is assigned a score for each ingredient, derived from their individual characteristics, and a label indicating whether they like, are neutral towards, or dislike the ingredient. There is also a random element added. While the influence of these factors is not precisely accurate and would require refinement in a real-world setting, they serve as a foundational basis for analysis in this research. The function also splits the data, which will be used to determine if the feedback can predict the rest of the children's preferences.

Reference: [Study on children's vegetable preferences](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6173934/)

## Add more references when writing


In [ ]:
import random
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Any, Tuple

def get_modifiers(
    features: Dict[str, Any],
    ingredient_row: pd.Series,
    health_consideration_modifiers: Dict[str, Dict[str, float]],
    favorite_cuisine_modifiers: Dict[str, Dict[str, float]],
    taste_modifiers: Dict[str, float],
    colour_modifiers: Dict[str, float],
    gender_modifiers: Dict[str, float],
    age_modifiers: Dict[int, float],
    texture_modifiers: Dict[str, float],
    other_modifiers: Dict[str, Any],
    vegetable_groups: Dict[str, list],
    group_probabilities_modifiers: Dict[str, float]
) -> float:
    health_consideration = features["health_consideration"]
    age = features["age"]
    gender = features["gender"]
    favorite_cuisine = features["favorite_cuisine"]

    health_category = ingredient_row["Healthy"]
    ingredient_category1 = ingredient_row["Category1"]
    taste = ingredient_row["Taste"]
    colour = ingredient_row["Colour"]
    texture = ingredient_row["Texture"]
    ingredient = ingredient_row["Category7"]

    health_mod = health_consideration_modifiers[health_consideration][health_category]
    favorite_mod = favorite_cuisine_modifiers.get(favorite_cuisine, {}).get(ingredient_category1, 1)
    taste_mod = taste_modifiers.get(taste, taste_modifiers["Misc"])
    colour_mod = colour_modifiers[colour]
    texture_mod = texture_modifiers[texture]
    gender_mod = gender_modifiers[gender]
    age_mod = age_modifiers[age]

    group_name = next((group for group, ingredients in vegetable_groups.items() if ingredient in ingredients), None)
    group_mod = group_probabilities_modifiers.get(group_name, 1)

    fruit_mod = other_modifiers["fruit_factor"] if ingredient_category1 == "Fruits and fruit products" else 1
    vegetable_mod = other_modifiers["vegetables_factor"][gender] if ingredient_category1 == "Vegetables and vegetable products" else 1
    meat_mod = other_modifiers["meat_factor"][gender] if ingredient_category1 == "Meat and meat products" else 1
    random_mod = random.uniform(other_modifiers["random_factor"][0], other_modifiers["random_factor"][1])

    return (health_mod * favorite_mod * taste_mod * colour_mod * gender_mod * age_mod * 
            texture_mod * group_mod * fruit_mod * vegetable_mod * meat_mod * random_mod)

def initialize_children_data(child_data: Dict[str, Dict[str, Any]], ingredient_df: pd.DataFrame, split: float = 0.8, seed: int = None, plot_graphs: bool = False) -> Tuple[Dict[str, Dict[str, list]], Dict[str, Dict[str, list]]]:
    random.seed(seed)
    children_data = {}
    all_scores = []
    all_preferences = {"likes": [], "neutral": [], "dislikes": []}

    # Factors affecting preferences with modifier values (increased impact)
    health_consideration_modifiers = {
        "very health conscious": {"healthy": 1.5, "average": 1, "unhealthy": 0.5},
        "moderately health conscious": {"healthy": 1.3, "average": 1, "unhealthy": 0.7},
        "don't care": {"healthy": 0.7, "average": 1, "unhealthy": 1.3},
    }

    favorite_cuisine_modifiers = {
        "BBQ": {"Meat and meat products": 1.4},
        "Seafood": {"Fish seafood amphibians reptiles and invertebrates": 1.4},
        "Italian": {"Anchovies": 1.4, "Aubergines": 1.4, "Noodles": 1.4, "Pasta plain (not stuffed) uncooked": 1.4, "Pasta wholemeal": 1.4, "Tomatoes": 1.4},
    }

    taste_modifiers = {
        "Sweet": 1.3,
        "Salty": 1.3,
        "Sour": 0.7,
        "Earthy": 0.7,
        "Misc": 1,
    }

    colour_modifiers = {
        "Red": 1.3,
        "Green": 1.3,
        "Yellow": 1.2,
        "Orange": 1.2,
        "Pink": 1,
        "Purple": 1,
        "White": 0.8,
        "Brown": 0.8,
    }

    gender_modifiers = {
        "M": 0.7,
        "F": 1.3,
    }

    age_modifiers = {
        9: 0.7,
        10: 1,
        11: 1.3,
    }

    texture_modifiers = {
        "Crunchy": 0.7,
        "Soft": 1.3,
        "Soft/Crunchy": 0.6,
        "Firm": 1.3,
        "Leafy": 1,
        "Grainy": 1,
        "Liquid": 1,
        "Powdery": 1,
        "Creamy": 1,
        "Hard": 1,
    }

    other_modifiers = {
        "fruit_factor": 1.3,
        "vegetables_factor": {"M": 0.7, "F": 1.3},
        "meat_factor": {"M": 1.3, "F": 0.7},
        "random_factor": [0.7, 1.3]
    }

    vegetable_groups = {
        "Group A": ["Tomatoes", "Sweet corn", "Sweet potatoes", "Carrots"],
        "Group B": ["Onions", "Spring onions", "Pepper"],
        "Group C": ["Cauliflowers"],
        "Group D": ["Courgettes", "Spinaches", "Curly kales", "Peas"],
        "Group E": ["Beetroots", "Lettuces (generic)", "Broccoli"],
        "Group F": ["Aubergines", "Cucumber", "White cabbage", "Savoy cabbages", "Red cabbage", "Runner beans (with pods)"],
    }

    group_probabilities_modifiers = {
        "Group A": 1.4,
        "Group B": 1.3,
        "Group C": 0.7, 
        "Group D": 1, 
        "Group E": 0.9, 
        "Group F": 0.9 
    }

    for child_key, features in child_data.items():
        preferences = {"likes": [], "neutral": [], "dislikes": []}
        child_scores = []

        for _, row in ingredient_df.iterrows():
            score = get_modifiers(features, row, health_consideration_modifiers, favorite_cuisine_modifiers, taste_modifiers,
                                  colour_modifiers, gender_modifiers, age_modifiers, texture_modifiers, other_modifiers,
                                  vegetable_groups, group_probabilities_modifiers)
            child_scores.append((row["Category7"], score))

        child_scores.sort(key=lambda x: x[1], reverse=True)
        all_scores.extend(child_scores)

        num_ingredients = len(child_scores)
        num_likes = int(0.6 * num_ingredients)
        num_neutral = int(0.2 * num_ingredients)
        num_dislikes = num_ingredients - num_likes - num_neutral

        preferences["likes"] = [ingredient for ingredient, _ in child_scores[:num_likes]]
        preferences["neutral"] = [ingredient for ingredient, _ in child_scores[num_likes:num_likes + num_neutral]]
        preferences["dislikes"] = [ingredient for ingredient, _ in child_scores[num_likes + num_neutral:]]

        all_preferences["likes"].extend(preferences["likes"])
        all_preferences["neutral"].extend(preferences["neutral"])
        all_preferences["dislikes"].extend(preferences["dislikes"])

        children_data[child_key] = preferences

    all_data = {}

    for child_key, preferences in children_data.items():
        known_preferences = {"likes": [], "neutral": [], "dislikes": []}
        unknown_preferences = {"likes": [], "neutral": [], "dislikes": []}

        for category in ["likes", "neutral", "dislikes"]:
            total_items = len(preferences[category])
            split_index = int(total_items * split)
            known_preferences[category] = preferences[category][:split_index]
            unknown_preferences[category] = preferences[category][split_index:]

        all_data[child_key] = {
            "known": known_preferences,
            "unknown": unknown_preferences
        }

    if plot_graphs:
        plot_histograms(all_scores, all_preferences)

    return all_data

def plot_histograms(scores: list, preferences: Dict[str, list]) -> None:
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.hist([score for ingredient, score in scores if ingredient in preferences["likes"]], bins=20, color='green', alpha=0.7, label='Like')
    plt.title('Like Scores')
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.hist([score for ingredient, score in scores if ingredient in preferences["neutral"]], bins=20, color='blue', alpha=0.7, label='Neutral')
    plt.title('Neutral Scores')
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.hist([score for ingredient, score in scores if ingredient in preferences["dislikes"]], bins=20, color='red', alpha=0.7, label='Dislike')
    plt.title('Dislike Scores')
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# from models.preferences.data_utils import get_child_data, initialize_children_data
from utils.process_data import get_data

ingredient_df = get_data()

all_preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False)

print(all_preferences)

### Child Feedback

The comments are designed to test the sentiment analysis capabilities of language models and determine if child preferences can be identified through feedback. The comments discuss multiple ingredients, allowing us to evaluate whether the sentiment towards each ingredient can be accurately measured through relatively simple sentiment analysis. Additionally, they help monitor how satisfaction with the meal plan will evolve over time. In reality, the comments wouldn't be as perfectly structured as they are here, but this serves as a good starting point. Feedback is provided based on the known and unknown preferences of the child. These comments will then be used to update the known preference list. 

In [ ]:
import numpy as np
import random
from typing import Dict

def get_feedback(preferences: Dict[str, Dict[str, list]], ingredient_list: list, seed=None):
    # Function to get feedback on meal plan which gives randomized comments on the ingredients for each child.
    # The function also sometimes doesn't provide feedback for some children. 
    comments = [
        ("Didn't like the {} and {} in the dish, but the {} was tasty.", ["dislikes", "dislikes", "likes"]),
        ("Did not enjoy the {} and {}.", ["dislikes", "dislikes"]),
        ("Enjoyed the {} and {}, but was okay with the {}.", ["likes", "likes", "neutral"]),
        ("Loved the {}, but didn't like the {} and {}.", ["likes", "dislikes", "dislikes"]),
        ("The {} was great, but the {} was average.", ["likes", "neutral"]),
        ("Didn't enjoy the {}, but the {} was average..", ["dislikes", "neutral"]),
        ("Loved the {} and {}, but not the {}.", ["likes", "likes", "dislikes"]),
        ("Loved the {}, but the {} was not appealing.", ["likes", "dislikes"]),
        ("Enjoyed the {}, but the {} was not liked.", ["likes", "dislikes"]),
        ("Didn't like the {}, {} and {} together.", ["dislikes", "dislikes", "dislikes"]),
        ("Really liked the {} with {} and the {} was tasty.", ["likes", "likes", "likes"]),
        ("Didn't like the {} in the dish, but the {} was fine.", ["dislikes", "neutral"]),
        ("Enjoyed the {} and {}, but not the {}.", ["likes", "likes", "dislikes"]),
        ("Didn't like the {} and {}.", ["dislikes", "dislikes"]),
        ("The {} and {} were amazing, but didn't enjoy the {} much.", ["likes", "likes", "dislikes"]),
        ("Loved the {} and {}, but not the {}.", ["likes", "likes", "dislikes"]),
        ("Didn't enjoy the {} much, but the {} was okay.", ["dislikes", "neutral"]),
        ("The {} and {} dish was great.", ["likes", "likes"]),
        ("Didn't like the {}.", ["dislikes"]),
        ("Enjoyed the {} and {}.", ["likes", "likes"]),
        ("Loved the {} and {}.", ["likes", "likes"]),
        ("Didn't like the {} and the {}.", ["dislikes", "dislikes"]),
        ("Enjoyed the {} and {}, but the {} was okay.", ["likes", "likes", "neutral"]),
        ("Didn't like the {} and {} in the dish.", ["dislikes", "dislikes"]),
        ("Didn't like the {}, but the {} was okay.", ["dislikes", "neutral"]),
        ("Enjoyed the {} and {}, but didn't like the {}.", ["likes", "likes", "dislikes"]),
        ("Didn't like the {}.", ["dislikes"]),
        ("Loved the {} and {}, but the {} was not liked.", ["likes", "likes", "dislikes"]),
        ("Didn't like the {}, but the {} was okay.", ["dislikes", "neutral"]),
        ("Enjoyed the {} and {}.", ["likes", "likes"]),
        ("Liked the {} but not the {}.", ["likes", "dislikes"]),
        ("The {} was fine, but the {} wasn't good.", ["neutral", "dislikes"]),
        ("The {} and {} were great, but the {} was not.", ["likes", "likes", "dislikes"]),
        ("The {} was tasty, but the {} wasn't.", ["likes", "dislikes"]),
        ("The {} was okay, but the {} wasn't appealing.", ["neutral", "dislikes"]),
        ("Didn't like the {}, but the {} was good.", ["dislikes", "likes"]),
        ("The {} and {} were okay, but the {} wasn't.", ["neutral", "neutral", "dislikes"]),
        ("Really liked the {}, but the {} was too strong.", ["likes", "dislikes"]),
        ("Enjoyed the {}, but the {} was too bland.", ["likes", "dislikes"]),
        ("The {} was fine, but the {} needed more flavor.", ["neutral", "dislikes"]),
        ("Loved the {}, but the {} was not good.", ["likes", "dislikes"]),
        ("Didn't enjoy the {}, but the {} was okay.", ["dislikes", "neutral"]),
        ("The {} was good, but the {} was not to my taste.", ["likes", "dislikes"]),
        ("Enjoyed the {}, but the {} was too overpowering.", ["likes", "dislikes"]),
        ("The {} was delicious, but the {} wasn't enjoyable.", ["likes", "dislikes"])
    ]
    

    random.seed(seed)
    feedback = {}

    for child, prefs in preferences.items():  # Iterate over each child and their preferences
        
        # Combine known and unknown preferences for likes, neutral, and dislikes
        available_ingredients = {
            "likes": prefs['known']['likes'] + prefs['unknown']['likes'],
            "neutral": prefs['known']['neutral'] + prefs['unknown']['neutral'],
            "dislikes": prefs['known']['dislikes'] + prefs['unknown']['dislikes'],
        }

        valid_comments = []  # Initialize the list of valid comments

        # Iterate over each comment template and its corresponding feedback types
        for comment_template, feedback_types in comments:
            matched_ingredients = []  # Initialize the list of matched ingredients
            used_ingredients = set()  # Initialize the set of used ingredients

            # Match ingredients according to feedback types
            for feedback_type in feedback_types:
                for category in available_ingredients:
                    if feedback_type in category:  # Check if the feedback type matches the category
                        # List possible ingredients not yet used
                        possible_ingredients = [ingredient for ingredient in ingredient_list if ingredient in available_ingredients[category] and ingredient not in used_ingredients]
                        if possible_ingredients:  # If there are possible ingredients
                            chosen_ingredient = random.choice(possible_ingredients)  # Randomly select an ingredient
                            matched_ingredients.append(chosen_ingredient)  # Add the chosen ingredient to the matched list
                            used_ingredients.add(chosen_ingredient)  # Mark the ingredient as used
                            break  # Break after finding a valid ingredient

            # Check if we have matched the required number of ingredients
            if len(matched_ingredients) == len(feedback_types):
                valid_comments.append((comment_template, matched_ingredients, feedback_types))  # Add to valid comments if matches found

        # Select a random valid comment from the list of valid comments
        if valid_comments:
            comment_template, matched_ingredients, feedback_types = random.choice(valid_comments)  # Randomly select a valid comment
            comment = comment_template.format(*matched_ingredients)  # Format the comment with matched ingredients
            correct_action = {ingredient: feedback_types[idx] for idx, ingredient in enumerate(matched_ingredients)}
            feedback[child] = {"comment": comment, "correct_action": correct_action}  # Add the comment to the child's feedback

    return feedback  # Return the feedback dictionary


In [ ]:
from models.preferences.data_utils import get_child_data, initialize_children_data, get_feedback
from utils.process_data import get_data

ingredient_df = get_data()
preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False, split = 0.2)

# print("Known:", preferences['child1']['known'])
# print("Unknown:", preferences['child1']['unknown'])

# # Example usage       # like             # dislike              # like                         # dislike                     # dislike
ingredient_list = ['Sweet potatoes', 'Rice grain brown', 'Cow ox or bull fresh meat', 'Wheat bread and rolls brown or wholemeal', 'Aubergines']
                # [like, dislike, like, like, dislike]
feedback = get_feedback(preferences, ingredient_list, seed=None)

for child, details in feedback.items():
    print(f"{child}: {details['comment']}")
    print(details['correct_action'])


##  Sentiment analysis

Use to take comments and update preferences, by finding what ingredients they speak positively about and don't 

In [12]:
import re
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def extract_preferences_and_update_data(preferences, feedback, ingredient_list, plot_confusion_matrix=False):
    changes = []
    incorrect_comments = []
    true_labels = []
    pred_labels = []
    
    # Mapping for confusion matrix
    label_mapping = {'likes': 0, 'neutral': 1, 'dislikes': 2}
    
    # Iterate over each child's feedback
    for child, fb in feedback.items():
        # Split comments into sentences based on punctuation
        comments = re.split(r'[,.!?]', fb["comment"].lower())
        correct_action = fb["correct_action"]
        
        # Analyze each comment's sentiment
        for sentence in comments:
            if sentence.strip():  # Check if the sentence is not empty
                pred_label = analyze_sentiment(sentence.strip())
                
                # Check for mentions of each ingredient in the sentence
                for ingredient in ingredient_list:
                    if ingredient.lower() in sentence:
                        change = {"child": child, "ingredient": ingredient, "change": ""}
                        
                        # Determine the appropriate category based on polarity
                        if pred_label == 'likes':  
                            if ingredient not in preferences[child]['known']["likes"]:
                                preferences[child]['known']["likes"].append(ingredient)
                                change["change"] = "added to likes"
                        elif pred_label == 'dislikes':
                            if ingredient not in preferences[child]['known']["dislikes"]:
                                preferences[child]['known']["dislikes"].append(ingredient)
                                change["change"] = "added to dislikes"
                        else:
                            pred_label = 'neutral'
                            if ingredient not in preferences[child]['known']["neutral"]:
                                preferences[child]['known']["neutral"].append(ingredient)
                                change["change"] = "added to neutral"
                        
                        # Remove ingredient from other lists
                        if change["change"]:
                            if change["change"] != "added to likes" and ingredient in preferences[child]['known']["likes"]:
                                preferences[child]['known']["likes"].remove(ingredient)
                                change["change"] += ", removed from likes"
                            if change["change"] != "added to dislikes" and ingredient in preferences[child]['known']["dislikes"]:
                                preferences[child]['known']['dislikes'].remove(ingredient)
                                change["change"] += ", removed from dislikes"
                            if change["change"] != "added to neutral" and ingredient in preferences[child]['known']["neutral"]:
                                preferences[child]['known']["neutral"].remove(ingredient)
                                change["change"] += ", removed from neutral"
                            
                            changes.append(change)
                        
                        true_labels.append(correct_action[ingredient])
                        pred_labels.append(pred_label)
                        
                        # Check if the prediction was incorrect
                        if pred_label != correct_action[ingredient]:
                            incorrect_comments.append({
                                "child": child,
                                "comment": sentence,
                                "predicted": pred_label,
                                "actual": correct_action[ingredient]
                            })

    # Calculate accuracy
    correct_actions = sum(1 for true, pred in zip(true_labels, pred_labels) if true == pred)
    total_actions = len(true_labels)
    accuracy = correct_actions / total_actions if total_actions > 0 else 0

    # Plot confusion matrix if flag is set
    if plot_confusion_matrix and total_actions > 0:
        cm = confusion_matrix([label_mapping[label] for label in true_labels], 
                              [label_mapping[label] for label in pred_labels], 
                              labels=[0, 1, 2])
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['likes', 'neutral', 'dislikes'])
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.show()

    return changes, preferences, accuracy, incorrect_comments


# Function to display the changes made to children's preferences
def display_changes(changes):
    for change in changes:
        print("Action Taken:\n")
        print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")
        
# Function to display incorrect comments and reasons
def display_incorrect_comments(incorrect_comments):
    for comment in incorrect_comments:
        print(f"Child {comment['child']} commented: '{comment['comment']}'")
        print(f"Predicted: {comment['predicted']}, Actual: {comment['actual']}\n")

### Text Blob Method

Fast but doesn't completley capture all the context around ingredients

In [ ]:
from textblob import TextBlob

# Function to analyze the sentiment of a comment
def analyze_sentiment(comment):
    # Create a TextBlob object from the comment
    analysis = TextBlob(comment)
    # Get the polarity of the comment's sentiment (-1 to 1)
    polarity = analysis.sentiment.polarity
    # Determine sentiment category
    if polarity > 0.1:
        return 'likes'
    elif polarity < -0.1:
        return 'dislikes'
    else:
        return 'neutral'


In [ ]:
from models.preferences.data_utils import get_child_data, initialize_children_data, get_feedback
from utils.process_data import get_data

ingredient_df = get_data()
preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False, split = 0.2)

# # Example usage       # like             # dislike              # like                         # dislike                     # dislike
ingredient_list = ['Sweet potatoes', 'Rice grain brown', 'Cow ox or bull fresh meat', 'Wheat bread and rolls brown or wholemeal', 'Aubergines']
                # [like, dislike, like, like, dislike]
feedback = get_feedback(preferences, ingredient_list, seed=None)

# Extract preferences and update data
changes, preferences, accuracy, incorrect_comments = extract_preferences_and_update_data(preferences, feedback, ingredient_list, plot_confusion_matrix=True)
print(f"Accuracy: {accuracy:.2f}")
# Display the changes
# for change in changes:
#     print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")

display_incorrect_comments(incorrect_comments)

### Vader

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Function to analyze the sentiment of a comment using VADER
def analyze_sentiment(comment):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(comment)
    
    # Remove the 'compound' score from the dictionary
    vs.pop('compound')

    # Determine sentiment category based on the highest probability
    sentiment = max(vs, key=vs.get)
    if sentiment == 'pos':
        return 'likes'
    elif sentiment == 'neg':
        return 'dislikes'
    else:
        return 'neutral'


In [ ]:
from models.preferences.data_utils import get_child_data, initialize_children_data, get_feedback
from utils.process_data import get_data

ingredient_df = get_data()
preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False, split = 0.2)

# # Example usage       # like             # dislike              # like                         # dislike                     # dislike
ingredient_list = ['Sweet potatoes', 'Rice grain brown', 'Cow ox or bull fresh meat', 'Wheat bread and rolls brown or wholemeal', 'Aubergines']
                # [like, dislike, like, like, dislike]
feedback = get_feedback(preferences, ingredient_list, seed=None)

# Extract preferences and update data
changes, preferences, accuracy, incorrect_comments = extract_preferences_and_update_data(preferences, feedback, ingredient_list, plot_confusion_matrix=True)
print(f"Accuracy: {accuracy:.2f}")
# Display the changes
# for change in changes:
#     print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")

display_incorrect_comments(incorrect_comments)



### Hugging Face Transformer Model

In [ ]:
from transformers import pipeline
import torch

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1
# Load the sentiment analysis pipeline with a specific model
sentiment_analyzer = pipeline('sentiment-analysis', model="finiteautomata/bertweet-base-sentiment-analysis", device=device)

def analyze_sentiment(comment):
    result = sentiment_analyzer(comment)
    label = result[0]['label']

    if label == 'POS':
        return 'likes'
    elif label == 'NEG':
        return 'dislikes'
    else:
        return 'neutral'

In [ ]:
from models.preferences.data_utils import get_child_data, initialize_children_data, get_feedback
from utils.process_data import get_data


ingredient_df = get_data()
preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False, split = 0.2)

# # Example usage       # like             # dislike              # like                         # dislike                     # dislike
ingredient_list = ['Sweet potatoes', 'Rice grain brown', 'Cow ox or bull fresh meat', 'Wheat bread and rolls brown or wholemeal', 'Aubergines']
                # [like, dislike, like, like, dislike]
feedback = get_feedback(preferences, ingredient_list, seed=None)

# Extract preferences and update data
changes, preferences, accuracy, incorrect_comments = extract_preferences_and_update_data(preferences, feedback, ingredient_list, plot_confusion_matrix=True)
print(f"Accuracy: {accuracy:.2f}")
# Display the changes
# for change in changes:
#     print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")

display_incorrect_comments(incorrect_comments)



## Collaborative Filtering

 ### Main Functions

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Define utility matrix function
def get_utility_matrix(children, ingredients, preferences):
    utility_matrix = np.zeros((len(children), len(ingredients)))
    for i, child in enumerate(children):
        for j, ingredient in enumerate(ingredients):
            if ingredient in preferences[child]['known']["likes"]:
                utility_matrix[i, j] = 1  # Likes are marked with 1
            elif ingredient in preferences[child]['known']["neutral"]:
                utility_matrix[i, j] = 0  # Neutral is marked with 0
            elif ingredient in preferences[child]['known']["dislikes"]:
                utility_matrix[i, j] = -1  # Dislikes are marked with -1
    return utility_matrix

# Define the function to create the known mask
def create_known_mask(target_child, ingredients, preferences):
    known_ingredients = (preferences[target_child]['known']['likes'] + 
                         preferences[target_child]['known']['neutral'] + 
                         preferences[target_child]['known']['dislikes'])
    known_mask = np.isin(ingredients, known_ingredients)
    return known_mask

# Define the user-based prediction function
def predict_preferences_user_based(target_child_index, user_similarity_matrix, utility_matrix, ingredients, preferences, clf):
    target_child = 'child' + str(target_child_index + 1)
    weighted_sum = np.dot(user_similarity_matrix[target_child_index], utility_matrix)
    sum_of_weights = np.sum(user_similarity_matrix[target_child_index])
    predicted_preferences = weighted_sum / sum_of_weights if sum_of_weights != 0 else np.zeros(len(ingredients))
    
    # Feature scaling
    scaler = StandardScaler()
    X = scaler.fit_transform(predicted_preferences.reshape(-1, 1))
    y = utility_matrix[target_child_index]
    
    
    known_mask = create_known_mask(target_child, ingredients, preferences)
    
    if np.sum(known_mask) < 2:  # Ensure we have at least two classes for training
        return np.full(len(ingredients), -1)
    
    clf.fit(X[known_mask], y[known_mask])
    predicted_classes = clf.predict(X)
    return predicted_classes

# Define the item-based prediction function
def predict_preferences_item_based(target_child_index, item_similarity_matrix, utility_matrix, ingredients, preferences, clf):
    target_child = 'child' + str(target_child_index + 1)
    # Get the preferences of the target child
    child_preferences = utility_matrix[target_child_index]
    
    # Calculate the weighted sum of item similarities for each item
    weighted_sum = np.dot(item_similarity_matrix, child_preferences)
    
    # Calculate the sum of weights for normalization
    sum_of_weights = np.sum(item_similarity_matrix, axis=1)
    
    # Normalize the weighted sum to get predicted preferences
    # Handle cases where sum_of_weights is zero
    predicted_preferences = np.divide(weighted_sum, sum_of_weights, out=np.zeros_like(weighted_sum), where=sum_of_weights != 0)
    
    # Replace NaNs with 0
    predicted_preferences = np.nan_to_num(predicted_preferences)
    
    # Feature scaling
    scaler = StandardScaler()
    X = scaler.fit_transform(predicted_preferences.reshape(-1, 1))
    y = utility_matrix[target_child_index]
    
    # Only use known preferences for training
    known_mask = create_known_mask(target_child, ingredients, preferences)

    if np.sum(known_mask) < 2:  # Ensure we have at least two classes for training
        return np.full(len(ingredients), -1)
    
    clf.fit(X[known_mask], y[known_mask])
    
    # Predict the classes
    predicted_classes = clf.predict(X)
    return predicted_classes
# Define the function to evaluate predictions
def evaluate_predictions(children, ingredients, preferences, utility_matrix, similarity_matrix, predict_preferences_function, clf, plot_confusion_matrix=False):
    class_map = {-1: 'dislikes', 0: 'neutral', 1: 'likes'}
    label_mapping = {1: 0, 0: 1, -1: 2}  # Map class labels to integers for confusion matrix
    total_correct_known = 0
    total_correct_unknown = 0
    total_incorrect = 0
    total_known = 0
    total_unknown = 0
    
    true_labels = []
    pred_labels = []
    
    for target_child_index in range(len(children)):
        child_key = 'child' + str(target_child_index + 1)
        predicted_classes = predict_preferences_function(target_child_index, similarity_matrix, utility_matrix, ingredients, preferences, clf)
        ingredient_ratings = {ingredient: rating for ingredient, rating in zip(ingredients, predicted_classes)}
        
        correct_known = 0
        correct_unknown = 0
        incorrect = 0
        known = 0
        unknown = 0

        known_preferences = set(preferences[child_key]['known']['likes']) | set(preferences[child_key]['known']['dislikes']) | set(preferences[child_key]['known']['neutral'])
        unknown_preferences = set(preferences[child_key]['unknown']['likes']) | set(preferences[child_key]['unknown']['dislikes']) | set(preferences[child_key]['unknown']['neutral'])
        
        print(f"\nPreferences for {child_key}:")
        for ingredient, rating in ingredient_ratings.items():
            is_correct = False
            true_label = None
            if ingredient in known_preferences:
                if ingredient in preferences[child_key]['known']["likes"]:
                    true_label = 1
                elif ingredient in preferences[child_key]['known']["neutral"]:
                    true_label = 0
                elif ingredient in preferences[child_key]['known']["dislikes"]:
                    true_label = -1
            elif ingredient in unknown_preferences:
                if ingredient in preferences[child_key]['unknown']["likes"]:
                    true_label = 1
                elif ingredient in preferences[child_key]['unknown']["neutral"]:
                    true_label = 0
                elif ingredient in preferences[child_key]['unknown']["dislikes"]:
                    true_label = -1

            true_labels.append(true_label)
            pred_labels.append(rating)

            if rating in class_map:
                if ingredient in preferences[child_key]['unknown'].get(class_map[rating], []):
                    print(f"{ingredient}: {rating}.  CORRECT, UNKNOWN")
                    correct_unknown += 1
                    unknown += 1
                    is_correct = True
                elif ingredient in preferences[child_key]['known'].get(class_map[rating], []):
                    print(f"{ingredient}: {rating}.  CORRECT, KNOWN")
                    correct_known += 1
                    known += 1
                    is_correct = True
            
            if not is_correct:
                print(f"{ingredient}: {rating}.  INCORRECT")
                incorrect += 1
                if ingredient in unknown_preferences:
                    unknown += 1
                elif ingredient in known_preferences:
                    known += 1

        total_predictions_child = correct_known + correct_unknown + incorrect
        accuracy = (correct_known + correct_unknown) / total_predictions_child if total_predictions_child > 0 else 0
        known_accuracy = (correct_known / known) if known > 0 else 0
        unknown_accuracy = (correct_unknown / unknown) if unknown > 0 else 0
        print(f"\nAccuracy Metrics for {child_key}:")
        print(f"Correct (Known): {correct_known} / {known} ({known_accuracy * 100:.2f}%)")
        print(f"Correct (Unknown): {correct_unknown} / {unknown} ({unknown_accuracy * 100:.2f}%)")
        print(f"Incorrect: {incorrect}")
        print(f"Total Predictions: {total_predictions_child}")
        print(f"Overall Accuracy: {accuracy:.2f}")
        
        total_correct_known += correct_known
        total_correct_unknown += correct_unknown
        total_incorrect += incorrect
        total_known += known
        total_unknown += unknown
    
    overall_total_predictions = total_correct_known + total_correct_unknown + total_incorrect
    overall_accuracy = (total_correct_known + total_correct_unknown) / overall_total_predictions if overall_total_predictions > 0 else 0
    overall_known_accuracy = (total_correct_known / total_known) if total_known > 0 else 0
    overall_unknown_accuracy = (total_correct_unknown / total_unknown) if total_unknown > 0 else 0
    print(f"\nOverall Accuracy Metrics:")
    print(f"Correct (Known): {total_correct_known} / {total_known} ({overall_known_accuracy * 100:.2f}%)")
    print(f"Correct (Unknown): {total_correct_unknown} / {total_unknown} ({overall_unknown_accuracy * 100:.2f}%)")
    print(f"Incorrect: {total_incorrect}")
    print(f"Total Predictions: {overall_total_predictions}")
    print(f"Overall Accuracy: {overall_accuracy:.2f}")

    # Plot confusion matrix if flag is set
    if plot_confusion_matrix and overall_total_predictions > 0:
        cm = confusion_matrix([label_mapping[label] for label in true_labels], 
                              [label_mapping[label] for label in pred_labels], 
                              labels=[0, 1, 2])
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['likes', 'neutral', 'dislikes'])
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.show()

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Classifier Models with tuned parameters
def get_classifier(classifier_name = 'RandomForest'):
    if classifier_name == 'LogisticRegression':
        solver = 'liblinear'
        class_weight = None
        C = 0.4022301244444916
        clf = LogisticRegression(solver=solver, class_weight=class_weight, C=C, max_iter=10000)
    elif classifier_name == 'RandomForest':
        n_estimators = 58
        max_depth = 11  
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    elif classifier_name == 'SVC':
        C = 3.5244469291048874
        kernel = 'rbf'
        clf = SVC(C=C, kernel=kernel, probability=True)
    elif classifier_name == 'SGDClassifier':
        alpha = 0.0018872854402943308
        max_iter = 3928  
        clf = SGDClassifier(alpha=alpha, max_iter=max_iter)
    return clf


# Best hyperparameters:  {'solver': 'lbfgs', 'class_weight': None, 'C': 6.2232422446177145}
# Best accuracy:  0.6775398303643858

# {'C': 3.5244469291048874, 'kernel': 'rbf'} SVC

# Best hyperparameters:  {'n_estimators': 58, 'max_depth': 11} RandomForest
# Best accuracy:  0.7418735389051784

# Best hyperparameters:  {'alpha': 0.0018872854402943308, 'max_iter': 3928}
# Best accuracy:  0.6574125625058961  (SGDClassifier)

Each entry in the item similarity matrix represents the cosine similarity between two ingredients. The value ranges from -1 to 1, where 1 indicates maximum similarity, 0 indicates no similarity, and -1 indicates maximum dissimilarity.

Higher similarity values between ingredients suggest that children who like (or dislike) one ingredient tend to have similar preferences for the other ingredient.

The predicted preferences array represents the estimated ratings a child would give to each ingredient, based on their current preferences and the similarities between ingredients.

Higher predicted preference values indicate a higher likelihood that the child will like the ingredient.

### User Based Collaborative Filtering

Concept: In user-based collaborative filtering, the idea is to find users who are similar to the target user based on their ratings and preferences. 

In [ ]:
# Assuming you have the functions get_child_data, get_data, and initialize_children_data
children = list(get_child_data().keys())  # Get the list of children
ingredient_df = get_data("data.csv")  # Get the ingredient data
ingredients = ingredient_df['Category7'].to_list()  # Convert ingredients to a list
preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False, split=0.5)  # Initialize children data with preferences

# Get the utility matrix based on updated preferences
utility_matrix = get_utility_matrix(children, ingredients, preferences)

# Calculate user similarity matrix
user_similarity_matrix = cosine_similarity(utility_matrix)

# Calculate item similarity matrix (using transposed utility matrix)
item_similarity_matrix = cosine_similarity(utility_matrix.T)

clf = get_classifier('RandomForest')
# Evaluate item-based predictions
print("\nUser-Based Predictions:")
evaluate_predictions(children, ingredients, preferences, utility_matrix, user_similarity_matrix, predict_preferences_user_based, clf, plot_confusion_matrix=True)


### Item Based Collaborative Filtering

Concept: In item-based collaborative filtering, the idea is to find items that are similar to the ones the target user has liked. Recommendations are then made based on the similarity between items.

In [ ]:
# Evaluate item-based predictions
print("\nItem-Based Predictions:")
evaluate_predictions(children, ingredients, preferences, utility_matrix, item_similarity_matrix, predict_preferences_item_based, clf, plot_confusion_matrix=True)


Item-based collaborative filtering with added features tends to be less sparse and more effective than user-based methods with added features because item features are stable, consistent, and richly described. These characteristics lead to a densely populated item similarity matrix, which can aggregate preferences more effectively and provide better coverage for recommendations. Additionally, item-based methods are more scalable and can handle sparse user interactions more gracefully, making them a preferred choice in many recommendation systems.

### Tuning

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import optuna
from sklearn.metrics import f1_score

# Define the function to evaluate predictions
def evaluate_predictions(children, ingredients, preferences, utility_matrix, similarity_matrix, predict_preferences_function, clf):
    total_correct_known = 0
    total_correct_unknown = 0
    total_incorrect = 0
    total_known = 0
    total_unknown = 0
    
    true_labels = []
    pred_labels = []
    
    for target_child_index in range(len(children)):
        child_key = "child" + str(target_child_index + 1)
        predicted_classes = predict_preferences_function(target_child_index, similarity_matrix, utility_matrix, ingredients, preferences, clf)
        ingredient_ratings = {ingredient: rating for ingredient, rating in zip(ingredients, predicted_classes)}
        
        correct_known = 0
        correct_unknown = 0
        incorrect = 0
        known = 0
        unknown = 0

        known_preferences = set(preferences[child_key]['known']['likes']) | set(preferences[child_key]['known']['dislikes']) | set(preferences[child_key]['known']['neutral'])
        unknown_preferences = set(preferences[child_key]['unknown']['likes']) | set(preferences[child_key]['unknown']['dislikes']) | set(preferences[child_key]['unknown']['neutral'])
        
        for ingredient, rating in ingredient_ratings.items():
            is_correct = False
            true_label = None
            if ingredient in known_preferences:
                if ingredient in preferences[child_key]['known']["likes"]:
                    true_label = 1
                elif ingredient in preferences[child_key]['known']["neutral"]:
                    true_label = 0
                elif ingredient in preferences[child_key]['known']["dislikes"]:
                    true_label = -1
            elif ingredient in unknown_preferences:
                if ingredient in preferences[child_key]['unknown']["likes"]:
                    true_label = 1
                elif ingredient in preferences[child_key]['unknown']["neutral"]:
                    true_label = 0
                elif ingredient in preferences[child_key]['unknown']["dislikes"]:
                    true_label = -1

            true_labels.append(true_label)
            pred_labels.append(rating)

            if ingredient in preferences[child_key]['unknown'].get('likes', []) and rating == 1:
                correct_unknown += 1
                unknown += 1
                is_correct = True
            elif ingredient in preferences[child_key]['unknown'].get('dislikes', []) and rating == -1:
                correct_unknown += 1
                unknown += 1
                is_correct = True
            elif ingredient in preferences[child_key]['unknown'].get('neutral', []) and rating == 0:
                correct_unknown += 1
                unknown += 1
                is_correct = True
            elif ingredient in preferences[child_key]['known'].get('likes', []) and rating == 1:
                correct_known += 1
                known += 1
                is_correct = True
            elif ingredient in preferences[child_key]['known'].get('dislikes', []) and rating == -1:
                correct_known += 1
                known += 1
                is_correct = True
            elif ingredient in preferences[child_key]['known'].get('neutral', []) and rating == 0:
                correct_known += 1
                known += 1
                is_correct = True
            
            if not is_correct:
                incorrect += 1
                if ingredient in unknown_preferences:
                    unknown += 1
                elif ingredient in known_preferences:
                    known += 1

        total_predictions_child = correct_known + correct_unknown + incorrect
        accuracy = (correct_known + correct_unknown) / total_predictions_child if total_predictions_child > 0 else 0
        known_accuracy = (correct_known / known) if known > 0 else 0
        unknown_accuracy = (correct_unknown / unknown) if unknown > 0 else 0
        
        total_correct_known += correct_known
        total_correct_unknown += correct_unknown
        total_incorrect += incorrect
        total_known += known
        total_unknown += unknown
    
    overall_total_predictions = total_correct_known + total_correct_unknown + total_incorrect
    overall_accuracy = (total_correct_known + total_correct_unknown) / overall_total_predictions if overall_total_predictions > 0 else 0
    overall_known_accuracy = (total_correct_known / total_known) if total_known > 0 else 0
    overall_unknown_accuracy = (total_correct_unknown / total_unknown) if total_unknown > 0 else 0
    
    # Compute the F1 score
    f1 = f1_score(true_labels, pred_labels, average='weighted')
    
    return f1
    

def objective(trial):
    
    preferences = initialize_children_data(get_child_data(), ingredient_df, seed=None, plot_graphs=False, split=0.5)  # Initialize children data with preferences
    
    # Define hyperparameters to tune
    classifier_names = ['LogisticRegression', 'RandomForest', 'SVC', 'SGDClassifier']
    classifier_name = classifier_names[3]
    # Choose between item-based or user-based recommendation
    recommendation_methods = ['item_based', 'user_based']
    recommendation_method = recommendation_methods[1]
    
    # Define classifier based on chosen name
    if classifier_name == 'LogisticRegression':
        solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear'])
        class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
        C = trial.suggest_float('C', 0.1, 10.0)
        clf = LogisticRegression(solver=solver, class_weight=class_weight, C=C, max_iter=10000)
    elif classifier_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    elif classifier_name == 'SVC':
        C = trial.suggest_float('C', 0.1, 10.0)
        kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        clf = SVC(C=C, kernel=kernel, probability=True)
    else:  # SGDClassifier
        alpha = trial.suggest_float('alpha', 0.00001, 0.1)
        max_iter = trial.suggest_int('max_iter', 1000, 10000)
        clf = SGDClassifier(alpha=alpha, max_iter=max_iter)
    
    # Get the utility matrix based on updated preferences
    utility_matrix = get_utility_matrix(children, ingredients, preferences)

    # Compute the appropriate similarity matrix
    if recommendation_method == 'item_based':
        item_similarity_matrix = cosine_similarity(utility_matrix.T)
        similarity_matrix = item_similarity_matrix
        predict_preferences_function = predict_preferences_item_based
    else:  # user_based
        user_similarity_matrix = cosine_similarity(utility_matrix)
        similarity_matrix = user_similarity_matrix
        predict_preferences_function = predict_preferences_user_based
    
    # Evaluate the model using the provided function
    f1 = evaluate_predictions(
        children, ingredients, preferences, utility_matrix, similarity_matrix, predict_preferences_function, clf
    )
    
    return f1

from models.preferences.data_utils import get_child_data, initialize_children_data
from utils.process_data import get_data

children = list(get_child_data().keys())  # Get the list of children
ingredient_df = get_data("data.csv")  # Get the ingredient data
ingredients = ingredient_df['Category7'].to_list()  # Convert ingredients to a list


# Create Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)


### ML Method

Use a classifier to predict if a child will like an ingredient based on all features

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, make_scorer
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

def get_data_preprocessor():
    preprocessor = ColumnTransformer(
        transformers=[
            ("age", OneHotEncoder(), ["age"]),
            ("gender", OneHotEncoder(), ["gender"]),
            ("health_consideration", OneHotEncoder(), ["health_consideration"]),
            ("favorite_cuisine", OneHotEncoder(), ["favorite_cuisine"]),
            ("type", OneHotEncoder(), ["type"]),
            ("colour", OneHotEncoder(), ["colour"]),
            ("taste", OneHotEncoder(), ["taste"]),
            ("texture", OneHotEncoder(), ["texture"]),
            ("healthy", OneHotEncoder(), ["healthy"]),
        ],
    )
    return preprocessor
# Function to prepare data for the machine learning model
def prepare_ml_data(preferences, ingredient_df, child_data, apply_SMOTE=False, seed=42):
    # Create a DataFrame from child_data
    child_df = pd.DataFrame.from_dict(child_data, orient='index').reset_index().rename(columns={'index': 'child'})
    
    # Create a DataFrame from preferences
    preferences_df = pd.DataFrame(preferences).T.reset_index().rename(columns={'index': 'child'})
    
    # Melt preferences DataFrame
    likes_df = preferences_df.explode('likes')[['child', 'likes']].rename(columns={'likes': 'ingredient'})
    likes_df['preference'] = 5
    
    neutral_df = preferences_df.explode('neutral')[['child', 'neutral']].rename(columns={'neutral': 'ingredient'})
    neutral_df['preference'] = 3
    
    dislikes_df = preferences_df.explode('dislikes')[['child', 'dislikes']].rename(columns={'dislikes': 'ingredient'})
    dislikes_df['preference'] = 1
    
    # Concatenate all preference DataFrames
    preferences_long_df = pd.concat([likes_df, neutral_df, dislikes_df])
    
    # Merge child data with preferences
    combined_df = pd.merge(child_df, preferences_long_df, on='child')
    
    # Merge the combined DataFrame with the ingredient DataFrame
    df = pd.merge(combined_df, ingredient_df[['Category7', 'Category1', 'Colour', 'Texture', 'Taste', 'Healthy']], left_on='ingredient', right_on='Category7')

    # Drop the redundant 'Category7' column after the merge
    df.drop(columns=['Category7'], inplace=True)

    # Rename columns to match the desired output
    df.rename(columns={
        'ingredient': 'ingredient',
        'Category1': 'type',
        'Colour': 'colour',
        'Texture': 'texture',
        'Taste': 'taste',
        'Healthy': 'healthy'
    }, inplace=True)
    
    # Select and reorder the final columns
    df = df[['age', 'gender', 'health_consideration', 'favorite_cuisine', 'ingredient', 'type', 'colour', 'texture', 'taste', 'healthy', 'preference']]
    
    # Encode the target variable
    label_encoder = LabelEncoder()
    df["preference"] = label_encoder.fit_transform(df["preference"])

    # Define the preprocessor for numerical and categorical features
    preprocessor = get_data_preprocessor()

    # Drop the target column before fitting the preprocessor, this will occur in column transformer but done to make it resilient to changes.
    X = df.drop(columns=['preference'])
    y = df["preference"].values

    # Fit the preprocessor
    preprocessor = preprocessor.fit(X)

    # Apply the transformations and prepare the dataset
    X = preprocessor.transform(X)
    
    if apply_SMOTE:
        # Convert sparse matrix to dense format
        X_dense = X.toarray() if hasattr(X, 'toarray') else X
        # Apply SMOTE to balance the classes
        smote = SMOTE(random_state=seed)
        X_res, y_res = smote.fit_resample(X_dense, y)
        
        # Convert back to DataFrame
        X_res_df = pd.DataFrame(X_res, columns=preprocessor.get_feature_names_out())
        y_res_df = pd.DataFrame(y_res, columns=['preference'])
        
        # Concatenate X and y
        df_res = pd.concat([X_res_df, y_res_df], axis=1)
        
        # Update df, X, and y
        df = df_res
        X = X_res
        y = y_res

    return X, y, df, label_encoder, preprocessor

def get_models():
    # Initialize the models
    models = {
        "Logistic Regression": LogisticRegression(max_iter=200),
        "Support Vector Machine": SVC(),
        "XGBoost": XGBClassifier(eval_metric='logloss'),
        "Random Forest": RandomForestClassifier(),
        "Gradient Boosting": GradientBoostingClassifier(),
        "AdaBoost": AdaBoostClassifier(algorithm='SAMME'),
        "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=1),
        "Decision Tree": DecisionTreeClassifier()
    }
    return models

def evaluate_models(X, y, n_splits=5):

    # Define scorers with zero_division=0 to handle undefined metric cases
    scorers = {
        'precision_macro': make_scorer(precision_score, average='macro', zero_division=0),
        'recall_macro': make_scorer(recall_score, average='macro', zero_division=0),
        'f1_macro': make_scorer(f1_score, average='macro', zero_division=0),
        'accuracy': make_scorer(accuracy_score)
    }

    # Evaluate models using cross-validation
    results = []
    accuracy_scores = {}
    for name, model in get_models().items():
        for scorer_name, scorer in scorers.items():
            score = cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=n_splits), scoring=scorer)
            results.append({
                "Model": name,
                "Metric": scorer_name,
                "Score": score.mean()
            })
            if scorer_name == 'accuracy':
                accuracy_scores[name] = score.mean()

    # Convert results to DataFrame for easy comparison
    results_df = pd.DataFrame(results)
    
    return results_df, accuracy_scores


def get_ranked_models(results_df):
    # Rank models for each metric
    ranked_results = results_df.copy()
    ranked_results['Rank'] = ranked_results.groupby('Metric')['Score'].rank(ascending=False, method='min')

    # Convert results to DataFrame for easy comparison
    results_df_sorted = ranked_results.sort_values(by=['Metric', 'Rank'])

    print(results_df_sorted)
    
def get_confusion_matrix(X, y, accuracy_scores, models, label_encoder):
    # Identify the best model based on accuracy
    best_model_name = max(accuracy_scores, key=accuracy_scores.get)
    best_model = models[best_model_name]

    # Fit the best model on the entire dataset and predict
    best_model.fit(X, y)
    y_pred = best_model.predict(X)

    # Plot confusion matrix
    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {best_model_name}')
    plt.show()
    

In [ ]:
from models.preferences.data_utils import initialize_children_data, get_child_data
from utils.process_data import get_data

ingredients_df = get_data("data.csv")
preferences = initialize_children_data(child_data=get_child_data(), ingredient_df=ingredients_df, seed=None, plot_graphs=False, split=0.5)
child_data = get_child_data()
models = get_models()

for child in preferences:
    ml_preferences[child] = {}
    ml_preferences[child]['likes'] = preferences[child]['known']['likes'] + preferences[child]['unknown']['likes']
    ml_preferences[child]['neutral'] = preferences[child]['known']['neutral'] + preferences[child]['unknown']['neutral']
    ml_preferences[child]['dislikes'] = preferences[child]['known']['dislikes'] + preferences[child]['unknown']['dislikes']

# # Prepare the data
X, y, df, label_encoder, preprocessor = prepare_ml_data(ml_preferences, ingredients_df, child_data)

results_df, accuracy_scores = evaluate_models(X, y)

get_ranked_models(results_df)

get_confusion_matrix(X, y, accuracy_scores, models, label_encoder)

Precision: Measures the accuracy of the positive predictions. High precision means that fewer false positives are present.

Recall: Measures the ability to capture all positive instances. High recall means that fewer false negatives are present.

F1 Score: The harmonic mean of precision and recall. It provides a balance between precision and recall.

### SMOTE
Popular data augmentation technique used to address class imbalance in datasets.

Identify Minority Class: First, SMOTE identifies the samples belonging to the minority class in the dataset.

Select a Sample: For each sample in the minority class, SMOTE selects one or more of its nearest neighbors (usually based on Euclidean distance) within the same class.

Generate Synthetic Samples: New synthetic samples are generated along the line segments joining the selected sample and its nearest neighbors. The position of the synthetic sample is determined by selecting a random point along the line segment. This ensures that the new samples are not just copies of existing ones but are rather slightly varied versions.

Repeat the Process: This process is repeated until the desired level of balance between the classes is achieved.

In [ ]:
from models.preferences.data_utils import initialize_children_data, get_child_data
from utils.process_data import get_data

ingredients_df = get_data("data.csv")
preferences = initialize_children_data(child_data=get_child_data(), ingredient_df=ingredients_df, seed=None, plot_graphs=False, split=0.5)
child_data = get_child_data()
models = get_models()

ml_preferences = {}

for child in preferences:
    ml_preferences[child] = {}
    ml_preferences[child]['likes'] = preferences[child]['known']['likes'] + preferences[child]['unknown']['likes']
    ml_preferences[child]['neutral'] = preferences[child]['known']['neutral'] + preferences[child]['unknown']['neutral']
    ml_preferences[child]['dislikes'] = preferences[child]['known']['dislikes'] + preferences[child]['unknown']['dislikes']
    
# # Prepare the data
X, y, df, label_encoder, preprocessor = prepare_ml_data(ml_preferences, ingredients_df, child_data)

# Convert sparse matrix to dense format if necessary
X_dense = X.toarray() if hasattr(X, 'toarray') else X

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_dense, y)

# Convert the transformed data back to a DataFrame for visualization
transformed_df = pd.DataFrame(X_resampled)

# Can increase n_splits as more data generated with smote
results_df, accuracy_scores = evaluate_models(X_resampled, y_resampled, n_splits=5)

get_ranked_models(results_df)

get_confusion_matrix(X_resampled, y_resampled, accuracy_scores, models, label_encoder)

Based on the results, it is evident that the machine learning method currently outperforms the collaborative filtering approach. However, it's important to note that with significantly larger datasets or improvements in data quality, the collaborative filtering method could potentially become more effective and surpass the performance of the machine learning models.

### Negotiation of preference
Now that we have estimated predictions for each ingredient based on various features, we can utilise this information to create an optimized meal plan that aims to satisfy every child. Additionally, we can incorporate other relevant features and stakeholder inputs into the process, feeding this enriched data into a reinforcement learning (RL) program to further refine and enhance the meal planning strategy.

Get trained model using all data

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Prepare the data
X, y, label_encoder, preprocessor = prepare_ml_data(preferences, ingredients_data, child_data)

# Train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X, y)

In [5]:
from models.preferences.data_utils import get_supplier_availability
from models.preferences.prediction import predict_preference_using_model

def collect_weighted_votes(preferences, ingredient_type, rf_model, preprocessor, child_data, ingredient_df, weight_function):
    supplier_availability = get_supplier_availability(ingredient_df)
    
    available_ingredients = {ingredient: 0 for ingredient, type in zip(ingredient_df['Category7'], ingredient_df['Category1']) if type == ingredient_type and supplier_availability.get(ingredient, False)}
    unavailable_ingredients = [ingredient for ingredient, available in supplier_availability.items() if not available and ingredient in ingredient_df['Category7'].values and ingredient_df[ingredient_df['Category7'] == ingredient]['Category1'].values[0] == ingredient_type]
    preference_to_rating_mapping = {0: 0, 1: 3, 2: 5}
    for child, pref in preferences.items():
        # Prepare child features for prediction
        child_features = {
            "age": child_data[child]["age"],
            "gender": child_data[child]["gender"],
            "health_consideration": child_data[child]["health_consideration"],
            "favorite_cuisine": child_data[child]["favorite_cuisine"]
        }
        
        # Calculate weights based on preferences and stability
        weights = weight_function(child, pref)
        
        likes = set(pref['known']["likes"])
        neutrals = set(pref['known']["neutral"])
        dislikes = set(pref['known']["dislikes"])
        
        # Update available ingredients' scores based on preferences and weights
        for ingredient in available_ingredients.keys():
            if ingredient in likes:
                available_ingredients[ingredient] += 5 * weights['likes']
            elif ingredient in neutrals:
                available_ingredients[ingredient] += 3 * weights['neutral']
            elif ingredient in dislikes:
                available_ingredients[ingredient] += 0 * weights['dislikes']
            else: # In the unknown list and have to predict
                # Predict preference for ingredients not explicitly liked or disliked
                ingredient_details = ingredient_df[ingredient_df['Category7'] == ingredient].iloc[0]
                ingredient_features = {
                    "ingredient": ingredient,
                    "type": ingredient_details["Category1"],
                    "texture": ingredient_details["Texture"],
                    "colour": ingredient_details["Colour"],
                    "taste": ingredient_details["Taste"],
                    "healthy": ingredient_details["Healthy"]
                }
                predicted_preference = predict_preference_using_model(child_features, ingredient_features, child, rf_model, preprocessor)
                
                if predicted_preference == 3:
                    pass
                
                available_ingredients[ingredient] += preference_to_rating_mapping[predicted_preference] * weights['dislikes']
            
    return available_ingredients, unavailable_ingredients

def negotiate_ingredients_simple(preferences, rf_model, preprocessor, child_data, ingredient_df, weight_function):
    ingredient_types = set(ingredient_df['Category1'])
    negotiated_ingredients = {}
    unavailable_ingredients = {}

    for ingredient_type in ingredient_types:
        votes, unavailable = collect_weighted_votes(preferences, ingredient_type, rf_model, preprocessor, child_data, ingredient_df, weight_function)
        sorted_ingredients = sorted(votes, key=votes.get, reverse=True)
        negotiated_ingredients[ingredient_type] = sorted_ingredients
        if unavailable:
            unavailable_ingredients[ingredient_type] = unavailable
    
    return negotiated_ingredients, unavailable_ingredients

def calculate_child_weight_simple(child, preferences):
    return {
        'likes': 1,
        'neutral': 1,
        'dislikes': 1
    }


In [19]:
def get_data_preprocessor():
    preprocessor = ColumnTransformer(
        transformers=[
            ("age", OneHotEncoder(), ["age"]),
            ("gender", OneHotEncoder(), ["gender"]),
            ("health_consideration", OneHotEncoder(), ["health_consideration"]),
            ("favorite_cuisine", OneHotEncoder(), ["favorite_cuisine"]),
            ("type", OneHotEncoder(), ["type"]),
            ("colour", OneHotEncoder(), ["colour"]),
            ("taste", OneHotEncoder(), ["taste"]),
            ("texture", OneHotEncoder(), ["texture"]),
            ("healthy", OneHotEncoder(), ["healthy"]),
        ],
    )
    return preprocessor

In [1]:
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import prince
import plotly.express as px
import random
from utils.process_data import get_data
from imblearn.over_sampling import SMOTE
import random
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Any, Tuple
import numpy as np
import random
from typing import Dict


 # Function to prepare data for the machine learning model
def prepare_ml_data(preferences, ingredient_df, child_data, apply_SMOTE=False, seed=42):
    # Create a DataFrame from child_data
    child_df = pd.DataFrame.from_dict(child_data, orient='index').reset_index().rename(columns={'index': 'child'})
    
    # Create a DataFrame from preferences
    preferences_df = pd.DataFrame(preferences).T.reset_index().rename(columns={'index': 'child'})
    
    # Melt preferences DataFrame
    likes_df = preferences_df.explode('likes')[['child', 'likes']].rename(columns={'likes': 'ingredient'})
    likes_df['preference'] = 5
    
    neutral_df = preferences_df.explode('neutral')[['child', 'neutral']].rename(columns={'neutral': 'ingredient'})
    neutral_df['preference'] = 3
    
    dislikes_df = preferences_df.explode('dislikes')[['child', 'dislikes']].rename(columns={'dislikes': 'ingredient'})
    dislikes_df['preference'] = 1
    
    unknown_df = preferences_df.explode('unknown')[['child', 'unknown']].rename(columns={'unknown': 'ingredient'})
    unknown_df['preference'] = np.nan
    
    # Concatenate all preference DataFrames
    preferences_long_df = pd.concat([likes_df, neutral_df, dislikes_df, unknown_df])
    
    # Merge child data with preferences
    combined_df = pd.merge(child_df, preferences_long_df, on='child')
    
    # Merge the combined DataFrame with the ingredient DataFrame
    df = pd.merge(combined_df, ingredient_df[['Category7', 'Category1', 'Colour', 'Texture', 'Taste', 'Healthy']], left_on='ingredient', right_on='Category7')

    # Drop the redundant 'Category7' column after the merge
    df.drop(columns=['Category7'], inplace=True)

    # Rename columns to match the desired output
    df.rename(columns={
        'ingredient': 'ingredient',
        'Category1': 'type',
        'Colour': 'colour',
        'Texture': 'texture',
        'Taste': 'taste',
        'Healthy': 'healthy'
    }, inplace=True)
    
    # Select and reorder the final columns
    df = df[['age', 'gender', 'health_consideration', 'favorite_cuisine', 'ingredient', 'type', 'colour', 'texture', 'taste', 'healthy', 'preference']]
    
    # Encode the target variable
    label_encoder = LabelEncoder()
    df["preference"] = label_encoder.fit_transform(df["preference"].astype(str))
    
    print(df["preference"].tolist())

    # Define the preprocessor for numerical and categorical features
    preprocessor = get_data_preprocessor()

    # Drop the target column before fitting the preprocessor
    X = df.drop(columns=['preference'])
    y = df["preference"].values

    # Fit the preprocessor
    preprocessor.fit(X)

    # Apply the transformations and prepare the dataset
    X_transformed = preprocessor.transform(X)

    # Drop rows with NaN preferences in the original DataFrame
    known_df = df.dropna(subset=['preference'])

    # Extract the indices of known preferences
    known_indices = known_df.index

    # Filter X_transformed and y to only include known preferences
    X = X_transformed[known_indices]
    y = y[known_indices]
    
    if apply_SMOTE:
        # Convert sparse matrix to dense format
        X_dense = X.toarray() if hasattr(X, 'toarray') else X
        # Apply SMOTE to balance the classes
        smote = SMOTE(random_state=seed)
        X_res, y_res = smote.fit_resample(X_dense, y)
        
        # Convert back to DataFrame
        X_res_df = pd.DataFrame(X_res, columns=preprocessor.get_feature_names_out())
        y_res_df = pd.DataFrame(y_res, columns=['preference'])
        
        # Concatenate X and y
        df_res = pd.concat([X_res_df, y_res_df], axis=1)
        
        # Update df, X, and y
        df = df_res
        X = X_res
        y = y_res

    return X, y, df, label_encoder, preprocessor


## Complete Workflow

+ Retrieve Data
+ Train predictor model
+ Negotiate Order
+ Retrieve Feedback
+ Update Preferences with Feedback
+ Update Predictor Model

+ Example Meal Plan: [onions, red pepper, egg plant, chicken, cauliflower, potatoes]

### Retrieving Data

In [2]:
from models.preferences.data_utils import initialize_children_data, get_child_data
from utils.process_data import get_data

ingredient_df = get_data("data.csv")
preferences = initialize_children_data(child_data=get_child_data(), ingredient_df=ingredient_df, seed=None, plot_graphs=False, split=0.8)
child_data = get_child_data()

Successfully read 136 lines from the file. Loaded 135 ingredients.


### Train Predictor Model

In [3]:
from models.preferences.data_utils import prepare_ml_data
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
ml_preferences = {}

for child in preferences:
    ml_preferences[child] = {}
    ml_preferences[child]['likes'] = preferences[child]['known']['likes']
    ml_preferences[child]['neutral'] = preferences[child]['known']['neutral']
    ml_preferences[child]['dislikes'] = preferences[child]['known']['dislikes']
    
    # Determine unknown features
    known_ingredients = set(preferences[child]['known']['likes'] + preferences[child]['known']['neutral'] + preferences[child]['known']['dislikes'])
    all_ingredients = set(ingredient_df['Category1'])
    unknown_ingredients = all_ingredients - known_ingredients
    
    ml_preferences[child]['unknown'] = list(unknown_ingredients)


# # Prepare the data
X, y, df, label_encoder, preprocessor = prepare_ml_data(ml_preferences, ingredient_df, get_child_data())

# Convert sparse matrix to dense format if necessary
X_dense = X.toarray() if hasattr(X, 'toarray') else X

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_dense, y)

# Train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X, y)


RandomForestClassifier()

### Negotiate order of preference

In [6]:
negotiated_ingredients, old_unavailable = negotiate_ingredients_simple(preferences, rf_model, preprocessor, child_data, ingredient_df, calculate_child_weight_simple)

print("Negotiated order of preferred ingredients by type:")
for ingredient_type, ingredients in negotiated_ingredients.items():
    print(f"{ingredient_type.capitalize()}: {', '.join(ingredients)}")

Negotiated order of preferred ingredients by type:
Grains and grain-based products: Couscous, Pasta wholemeal, Rice grain, Pasta plain (not stuffed) uncooked, Noodles, Rice grain brown, Rice grain long-grain, Wheat bread and rolls brown or wholemeal, Wheat bread and rolls white (refined flour), Wheat flour white, Buckwheat
Animal and vegetable fats and oils and primary derivatives thereof: Sunflower seed oil edible, Rape seed oil edible, Traditional margarine, Palm oil/fat, Butter
Legumes nuts oilseeds and spices: Peas, Kidney bean, Coconuts, Chickpeas (dry), Pistachios, Pumpkin seeds, Brazil nuts, Lentils (dry), Almonds, Cashew nuts, Peanuts, Hazelnuts
Eggs: Eggs
Vegetables and vegetable products: Carrots, Swedes, Sweet corn, Tomatoes, Watermelons, Chili peppers, Asparagus, Courgettes, Cucumber, Leeks, Corianders, Basil, Rosemary, Parsley, Oregano, Aubergines, Rhubarbs, Brussels sprouts, Onions, Spring onions, Garlic, Parsnip roots, Broccoli, Celeries, Common bean sprouts, Runner bean

### Get Feedback

In [7]:
from models.preferences.data_utils import get_feedback, initialize_children_data, get_child_data
from utils.process_data import get_data
import random

ingredient_df = get_data("data.csv")
menu_plan = random.sample(list(ingredient_df['Category7']), 5)

preferences = initialize_children_data(get_child_data(), ingredient_df)

print(menu_plan)

feedback = get_feedback(preferences, menu_plan)

Successfully read 136 lines from the file. Loaded 135 ingredients.
['Pepper', 'Watermelons', 'Oregano', 'Brazil nuts', 'Runner beans (with pods)']


### Update Preferences with Feedback

In [8]:
import re
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def extract_preferences_and_update_data(preferences, feedback, menu_plan, plot_confusion_matrix=False):
    changes = []
    incorrect_comments = []
    true_labels = []
    pred_labels = []
    
    # Mapping for confusion matrix
    label_mapping = {'likes': 5, 'neutral': 3, 'dislikes': 0}

    # Check if GPU is available and set device accordingly
    device = 0 if torch.cuda.is_available() else -1
    # Load the sentiment analysis pipeline with a specific model
    sentiment_analyzer = pipeline('sentiment-analysis', model="finiteautomata/bertweet-base-sentiment-analysis", device=device)
    
    # Iterate over each child's feedback
    for child, fb in feedback.items():
        # Split comments into sentences based on punctuation
        comments = re.split(r'[,.!?]', fb["comment"].lower())
        correct_action = fb["correct_action"]
        
        # Analyze each comment's sentiment
        for sentence in comments:
            if sentence.strip():  # Check if the sentence is not empty
                comment = sentence.strip()
                pred_label = analyze_sentiment(comment, sentiment_analyzer)
                
                # Check for mentions of each ingredient in the sentence
                for ingredient in menu_plan:
                    if ingredient.lower() in sentence:
                        change = {"child": child, "ingredient": ingredient, "change": ""}
                        
                        # Determine the appropriate category based on polarity
                        if pred_label == 'likes':  
                            if ingredient not in preferences[child]['known']["likes"]:
                                preferences[child]['known']["likes"].append(ingredient)
                                change["change"] = "added to likes"
                        elif pred_label == 'dislikes':
                            if ingredient not in preferences[child]['known']["dislikes"]:
                                preferences[child]['known']["dislikes"].append(ingredient)
                                change["change"] = "added to dislikes"
                        else:
                            pred_label = 'neutral'
                            if ingredient not in preferences[child]['known']["neutral"]:
                                preferences[child]['known']["neutral"].append(ingredient)
                                change["change"] = "added to neutral"
                        
                        # Remove ingredient from other lists
                        if change["change"]:
                            if change["change"] != "added to likes" and ingredient in preferences[child]['known']["likes"]:
                                preferences[child]['known']["likes"].remove(ingredient)
                                change["change"] += ", removed from likes"
                            if change["change"] != "added to dislikes" and ingredient in preferences[child]['known']["dislikes"]:
                                preferences[child]['known']['dislikes'].remove(ingredient)
                                change["change"] += ", removed from dislikes"
                            if change["change"] != "added to neutral" and ingredient in preferences[child]['known']["neutral"]:
                                preferences[child]['known']["neutral"].remove(ingredient)
                                change["change"] += ", removed from neutral"
                            
                            changes.append(change)
                        
                        true_labels.append(correct_action[ingredient])
                        pred_labels.append(pred_label)
                        
                        # Check if the prediction was incorrect
                        if pred_label != correct_action[ingredient]:
                            incorrect_comments.append({
                                "child": child,
                                "comment": sentence,
                                "predicted": pred_label,
                                "actual": correct_action[ingredient]
                            })

    # Calculate accuracy
    correct_actions = sum(1 for true, pred in zip(true_labels, pred_labels) if true == pred)
    total_actions = len(true_labels)
    accuracy = correct_actions / total_actions if total_actions > 0 else 0

    # Plot confusion matrix if flag is set
    if plot_confusion_matrix and total_actions > 0:
        cm = confusion_matrix([label_mapping[label] for label in true_labels], 
                              [label_mapping[label] for label in pred_labels], 
                              labels=[0, 1, 2])
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['likes', 'neutral', 'dislikes'])
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.show()

    return changes, preferences, accuracy, incorrect_comments


# Function to display the changes made to children's preferences
def display_changes(changes):
    for change in changes:
        print("Action Taken:\n")
        print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")
        
# Function to display incorrect comments and reasons
def display_incorrect_comments(incorrect_comments):
    for comment in incorrect_comments:
        print(f"Child {comment['child']} commented: '{comment['comment']}'")
        print(f"Predicted: {comment['predicted']}, Actual: {comment['actual']}\n")

In [10]:
from transformers import pipeline
import torch

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1
# Load the sentiment analysis pipeline with a specific model
sentiment_analyzer = pipeline('sentiment-analysis', model="finiteautomata/bertweet-base-sentiment-analysis", device=device)

def analyze_sentiment(comment, sentiment_analyzer):
    result = sentiment_analyzer(comment)
    label = result[0]['label']

    if label == 'POS':
        return 'likes'
    elif label == 'NEG':
        return 'dislikes'
    else:
        return 'neutral'

# Update children_data and get the list of changes
changes, updated_preferences, accuracy, incorrect_comments = extract_preferences_and_update_data(preferences, feedback, menu_plan)

# Function to display the changes made to children's preferences
def display_changes(changes):
    for change in changes:
        print("Action Taken:\n")
        print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")
        
# Function to display incorrect comments and reasons
def display_incorrect_comments(incorrect_comments):
    for comment in incorrect_comments:
        print(f"Child {comment['child']} commented: '{comment['comment']}'")
        print(f"Predicted: {comment['predicted']}, Actual: {comment['actual']}\n")

# # Display the changes
display_changes(changes)

display_incorrect_comments(incorrect_comments)

print(accuracy)

2024-07-15 19:36:33.859298: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 19:36:33.908895: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 19:36:34.721331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Action Taken:

Child child1 had Runner beans (with pods) added to neutral, removed from dislikes, removed from neutral.
Action Taken:

Child child3 had Oregano added to likes.
Action Taken:

Child child4 had Brazil nuts added to dislikes.
Action Taken:

Child child5 had Runner beans (with pods) added to dislikes.
Action Taken:

Child child6 had Runner beans (with pods) added to likes.
Action Taken:

Child child6 had Brazil nuts added to dislikes.
Action Taken:

Child child7 had Brazil nuts added to neutral, removed from dislikes, removed from neutral.
Action Taken:

Child child8 had Runner beans (with pods) added to likes.
Action Taken:

Child child9 had Brazil nuts added to dislikes.
Action Taken:

Child child10 had Brazil nuts added to dislikes.
Action Taken:

Child child11 had Runner beans (with pods) added to likes.
Action Taken:

Child child12 had Oregano added to likes.
Action Taken:

Child child13 had Brazil nuts added to dislikes.
Action Taken:

Child child18 had Pepper added t

In [9]:
print(updated_preferences)
print(preferences)

NameError: name 'updated_preferences' is not defined

### Retrain predictor model with updated preferences

In [11]:
from models.preferences.data_utils import get_child_data, prepare_ml_data
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
ml_preferences = {}

for child in updated_preferences:
    ml_preferences[child] = {}
    ml_preferences[child]['likes'] = updated_preferences[child]['known']['likes']
    ml_preferences[child]['neutral'] = updated_preferences[child]['known']['neutral']
    ml_preferences[child]['dislikes'] = updated_preferences[child]['known']['dislikes']
    
    # Determine unknown features
    known_ingredients = set(updated_preferences[child]['known']['likes'] + updated_preferences[child]['known']['neutral'] + updated_preferences[child]['known']['dislikes'])
    all_ingredients = set(ingredient_df['Category1'])
    unknown_ingredients = all_ingredients - known_ingredients
    
    ml_preferences[child]['unknown'] = list(unknown_ingredients)


# # Prepare the data
X, y, df, label_encoder, preprocessor = prepare_ml_data(ml_preferences, ingredient_df, get_child_data())

# Convert sparse matrix to dense format if necessary
X_dense = X.toarray() if hasattr(X, 'toarray') else X

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_dense, y)

# Train the Random Forest model
rf_model = RandomForestClassifier()

rf_model.fit(X, y)


RandomForestClassifier()

### New supplier availability list

### Renegotiate order of preference

In [17]:
renegotiated_ingredients, new_unavailable = negotiate_ingredients_simple(updated_preferences, rf_model, preprocessor, get_child_data(), ingredient_df, calculate_child_weight_simple)

print("Negotiated order of preferred ingredients by type:")
for ingredient_type, ingredients in negotiated_ingredients.items():
    print(f"{ingredient_type.capitalize()}: {', '.join(ingredients)}")

Negotiated order of preferred ingredients by type:
Grains and grain-based products: Couscous, Pasta wholemeal, Rice grain, Pasta plain (not stuffed) uncooked, Noodles, Rice grain brown, Rice grain long-grain, Wheat bread and rolls brown or wholemeal, Wheat bread and rolls white (refined flour), Wheat flour white, Buckwheat
Animal and vegetable fats and oils and primary derivatives thereof: Sunflower seed oil edible, Rape seed oil edible, Traditional margarine, Palm oil/fat, Butter
Legumes nuts oilseeds and spices: Peas, Kidney bean, Coconuts, Chickpeas (dry), Pistachios, Pumpkin seeds, Brazil nuts, Lentils (dry), Almonds, Cashew nuts, Peanuts, Hazelnuts
Eggs: Eggs
Vegetables and vegetable products: Carrots, Swedes, Sweet corn, Tomatoes, Watermelons, Chili peppers, Asparagus, Courgettes, Cucumber, Leeks, Corianders, Basil, Rosemary, Parsley, Oregano, Aubergines, Rhubarbs, Brussels sprouts, Onions, Spring onions, Garlic, Parsnip roots, Broccoli, Celeries, Common bean sprouts, Runner bean

### Compare two negotiated lists to see what has changed

In [19]:
def generate_ordered_list(ingredients_list):
    return {ingredient: index + 1 for index, ingredient in enumerate(ingredients_list)}

def compare_negotiated_ingredients(old_ingredients, new_ingredients, old_unavailable, new_unavailable):
    changes = {}

    all_ingredient_types = set(old_ingredients.keys()).union(new_ingredients.keys()).union(old_unavailable.keys()).union(new_unavailable.keys())

    for ingredient_type in all_ingredient_types:
        old_list = old_ingredients.get(ingredient_type, [])
        new_list = new_ingredients.get(ingredient_type, [])
        old_unavail_list = old_unavailable.get(ingredient_type, [])
        new_unavail_list = new_unavailable.get(ingredient_type, [])

        old_order = generate_ordered_list(old_list)
        new_order = generate_ordered_list(new_list)

        order_changes = []
        for ingredient in set(old_order.keys()).union(new_order.keys()).union(old_unavail_list).union(new_unavail_list):
            old_pos = old_order.get(ingredient, 'Unavailable')
            new_pos = new_order.get(ingredient, "Error")
            
            if ingredient in old_unavail_list:
                old_pos = 'Unavailable'
            if ingredient in new_unavail_list:
                new_pos = 'Unavailable'
                
            if old_pos != new_pos:
                order_changes.append((ingredient, old_pos, new_pos))

        if order_changes:
            changes[ingredient_type] = order_changes

    return changes

In [20]:
changes = compare_negotiated_ingredients(negotiated_ingredients, renegotiated_ingredients, old_unavailable, new_unavailable)

# Display the changes in a readable format
for ingredient_type, order_changes in changes.items():
    print(f"Changes in {ingredient_type}:")
    for ingredient, old_pos, new_pos in order_changes:
        print(f"{ingredient}: Pos: {old_pos} -> Pos: {new_pos}")

Changes in Grains and grain-based products:
Rice grain: Pos: 3 -> Pos: Unavailable
Wheat wholemeal flour: Pos: Unavailable -> Pos: 10
Pasta plain (not stuffed) uncooked: Pos: 4 -> Pos: 5
Noodles: Pos: 5 -> Pos: 3
Wheat bread and rolls brown or wholemeal: Pos: 8 -> Pos: 6
Wheat bread and rolls white (refined flour): Pos: 9 -> Pos: 8
Rice grain brown: Pos: 6 -> Pos: 4
Wheat flour white: Pos: 10 -> Pos: 9
Changes in Animal and vegetable fats and oils and primary derivatives thereof:
Olive oil: Pos: Unavailable -> Pos: 3
Traditional margarine: Pos: 3 -> Pos: 6
Changes in Vegetables and vegetable products:
Cauliflowers: Pos: 31 -> Pos: 28
Turnips: Pos: 33 -> Pos: 32
Garlic: Pos: 21 -> Pos: Unavailable
Spinaches: Pos: 28 -> Pos: 25
Oregano: Pos: 15 -> Pos: 16
Parsley: Pos: 14 -> Pos: 13
Lettuces (generic): Pos: 32 -> Pos: 26
Radishes: Pos: 29 -> Pos: 24
Swedes: Pos: 2 -> Pos: 6
Spring onions: Pos: 20 -> Pos: 21
White cabbage: Pos: 37 -> Pos: 38
Common bean sprouts: Pos: 25 -> Pos: 27
Savoy c

# Fairness ?

### Fairness Principles
**Equity Principle**: We integrated the equity principle by considering the impact of a child's vote through the number of preferences.

The initial method for weighting votes relied on the total number of preferences (likes, neutrals, and dislikes) each child expressed. This approach can unintentionally give more influence to children with a higher number of preferences. To ensure a fairer distribution of votes, we should adjust the weights of the votes more thoughtfully, considering both the type of preferences and the number of preferences each child has.

To achieve this, votes will be normalized so that each child's dislike, neutral, and like votes have a similar impact. For example, if one child dislikes 20 ingredients while another child dislikes only 2, the child with fewer dislikes would have a disproportionately small dislike impact if the votes were not adjusted with a weighting factor. Therefore, it is prudent to make the impact of one child’s 20 dislikes more comparable to the impact of another child’s 2 dislikes, ensuring each child's preferences are equitably represented.

**Need Principle**: 
+ Firstly, This principle is addressed by categorizing ingredients by type, ensuring that an ingredient from each group is chosen. 
+ Secondly, in the top 10% of the list no child will have all their disliked ingredients contained within. Otherwise, their basic needs will not be met, as they will continually receive items they dislike. This prevents any one child not meeting nutritional targets.

**Meritocracy Principle**

+ To incorporate the principle of meritocracy, we must define actions that contribute to our collective goal: providing healthier, cheaper, and improved nutrition in school meals.
+ The children help achieve this by reducing food waste and improving their preferences.

Therefore, to give merit to good actions:

1. **Proportional Voting Weight**:
   - The weight of a child's vote will be proportional to their like/dislike ratio. For example, if a child dislikes 10 ingredients but likes only 5, their vote will account for 50% less compared to a child who likes and dislikes an equal number of ingredients.

2. **Feedback Participation**:
   - Children who actively participate in giving feedback on previous meals will have increased voting power. Conversely, those who do not provide feedback will have reduced voting power for their preferences.

3. **Merit Points for Good Behavior**:
   - Children can earn merit points for positive behaviors such as reducing food waste. More merit points will translate to a more powerful vote.

**Transparency and Communication**

+ The system must be transparent. Ensure that the criteria for earning merit points and the conversion of these points to voting power are clearly communicated and understood by all participants.

+ By implementing these measures, we aim to create a fair and motivating environment that encourages children to contribute positively towards our collective goal of improving school meals.


In [ ]:
child_data = get_child_data()
ingredients_data = get_ingredient_data()
preferences = get_preference_data()

In [ ]:
def calculate_child_weight(child, preferences):
    
    # Normalize likes, neutrals, and dislikes so each child has equal contribution
    total_likes = len(preferences["likes"])
    total_neutrals = len(preferences["neutral"])
    total_dislikes = len(preferences["dislikes"])
    
    # Make vote proportional to like/dislike ratio
    like_dislike_ratio = total_likes / total_dislikes
    
    child_merit_weight = 1  # Placeholder for child-specific merit
    
    if str(child) in get_feedback().keys():
        feedback_provided_weight = 1
    else:
        feedback_provided_weight = 0.9
    
    # Ensure each category contributes equally
    normalized_likes = 1 / max(1, total_likes)
    normalized_neutrals = 1 / max(1, total_neutrals)
    normalized_dislikes = 1 / max(1, total_dislikes)
    
    return {
        'likes': normalized_likes * like_dislike_ratio * child_merit_weight * feedback_provided_weight,
        'neutral': normalized_neutrals * like_dislike_ratio * child_merit_weight * feedback_provided_weight,
        'dislikes': normalized_dislikes * like_dislike_ratio * child_merit_weight * feedback_provided_weight
    }

def collect_weighted_votes(preferences, ingredient_type, rf_model, preprocessor, child_data, ingredients_data):
    supplier_availability = get_supplier_availability()
    
    available_ingredients = {ingredient: 0 for ingredient, details in ingredients_data.items() if details['type'] == ingredient_type and supplier_availability.get(ingredient, False)}
    unavailable_ingredients = [ingredient for ingredient, available in supplier_availability.items() if not available and ingredient in ingredients_data and ingredients_data[ingredient]['type'] == ingredient_type]
    
    for child, pref in preferences.items():
        
        # Prepare child features for prediction
        child_features = {
            "age": child_data[child]["age"],
            "gender": 1 if child_data[child]["gender"] == "M" else 0,
            "health_consideration": child_data[child]["health_consideration"],
            "favorite_cuisine": child_data[child]["favorite_cuisine"]
        }
        
        # Calculate weights based on preferences and stability
        weights = calculate_child_weight(child, pref)
        
        likes = set(pref["likes"])
        neutrals = set(pref["neutral"])
        dislikes = set(pref["dislikes"])
        
        # Update available ingredients' scores based on preferences and weights
        for ingredient in available_ingredients.keys():
            if ingredient in likes:
                available_ingredients[ingredient] += 5 * weights['likes']
            elif ingredient in neutrals:
                available_ingredients[ingredient] += 3 * weights['neutral']
            elif ingredient not in dislikes:
                # Predict preference for ingredients not explicitly liked or disliked
                ingredient_details = ingredients_data[ingredient]
                ingredient_features = {
                    "ingredient": ingredient,
                    "type": ingredient_details["type"],
                    "color": ingredient_details["color"],
                    "taste": ingredient_details["taste"]
                }
                predicted_preference = predict_preference(child_features, ingredient_features, rf_model, preprocessor)
                available_ingredients[ingredient] += predicted_preference * weights['dislikes']
    
    return available_ingredients, unavailable_ingredients

def negotiate_ingredients(preferences, rf_model, preprocessor, child_data, ingredients_data):
    ingredient_types = set(details['type'] for details in ingredients_data.values())
    negotiated_ingredients = {}
    unavailable_ingredients = {}

    for ingredient_type in ingredient_types:
        votes, unavailable = collect_weighted_votes(preferences, ingredient_type, rf_model, preprocessor, child_data, ingredients_data)
        
        # Sort ingredients based on votes
        sorted_ingredients = sorted(votes, key=votes.get, reverse=True)
        top_10_percent = sorted_ingredients[:max(1, len(sorted_ingredients) // 10)]
        
        # Check top 10% and ensure no child has all their dislikes in this section
        for child, pref in preferences.items():
            dislikes_in_top_10 = set(pref['dislikes']).intersection(top_10_percent)
            if len(dislikes_in_top_10) == len(top_10_percent):
                # If all top 10% are disliked by one child, reassign scores to balance
                for dislike in dislikes_in_top_10:
                    votes[dislike] -= 1  # Penalize disliked items slightly
                    sorted_ingredients = sorted(votes, key=votes.get, reverse=True)
                    top_10_percent = sorted_ingredients[:max(1, len(sorted_ingredients) // 10)]
        
        negotiated_ingredients[ingredient_type] = sorted_ingredients
        
        # Track unavailable ingredients
        if unavailable:
            unavailable_ingredients[ingredient_type] = unavailable
    
    return negotiated_ingredients, unavailable_ingredients



negotiated_ingredients_resher, unavailable_ingredients_resher = negotiate_ingredients(preferences, rf_model, preprocessor, child_data, ingredients_data)

print("Negotiated Ingredients:", negotiated_ingredients)
print("Unavailable Ingredients:", unavailable_ingredients)

In [ ]:
changes = compare_negotiated_ingredients(negotiated_ingredients_after_feedback, negotiated_ingredients_resher, new_unavailable, unavailable_ingredients_resher)

# Display the changes in a readable format
for ingredient_type, order_changes in changes.items():
    print(f"Changes in {ingredient_type}:")
    for ingredient, old_pos, new_pos in order_changes:
        print(f"{ingredient}: Pos: {old_pos} -> Pos: {new_pos}")

In [ ]:
import plotly.express as px
# Function to plot 3D MCA components interactively
def plot_3d_mca_interactive(X, y):
    # Ensure we only take the first three components
    if X.shape[1] < 3:
        raise ValueError("Input data must have at least 3 principal components for a 3D plot.")
    
    # Create a DataFrame with the first three MCA components and the labels
    df = X.iloc[:, :3]
    df.columns = ['MC1', 'MC2', 'MC3'] 
    df['Preference'] = y
    
    # Create an interactive 3D scatter plot
    fig = px.scatter_3d(df, x='MC1', y='MC2', z='MC3', color='Preference', 
                        title='3D MCA Interactive Plot', labels={'Preference': 'Preference'})
    fig.show()

In [ ]:
# Function to plot scree plot for MCA
def plot_scree_mca(preprocessor, n_components=10):
    # Extract the MCA step from the pipeline
    # Extract the explained inertia from the MCA step
    mca_step = pipeline.named_steps['mca']
    explained_inertia = mca_step.explained_inertia_
    # Plot the explained variance
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, n_components + 1), explained_inertia, marker='o', linestyle='--')
    plt.title('Scree Plot')
    plt.xlabel('MCA Component')
    plt.ylabel('Explained Inertia Ratio')
    plt.xticks(range(1, n_components + 1))
    plt.grid()
    plt.show()

In [ ]:
ingredient_df = get_ingredient_data()
preferences = get_preference_data()
child_data = get_child_data()
X, y, label_encoder, preprocessor = prepare_ml_data(preferences, ingredient_df, child_data)

# Convert sparse matrix to dense format if necessary
X_dense = X.toarray() if hasattr(X, 'toarray') else X

# Apply SMOTE
smote = SMOTE(random_state=42)

X_resampled, y_resampled = smote.fit_resample(X_dense, y)

print(X_resampled.shape)
# from sklearn.manifold import TSNE
# tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
# tsne_components = tsne.fit_transform(one_hot_encoded_df)


# plot_3d_mca(X, y)
# plot_2d_mca(X, y)
# plot_3d_mca_interactive(X, y)
# plot_scree_mca(preprocessor, n_components=10)

In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px

# # Assuming X is your feature DataFrame and y is your target Series
tsne = TSNE(n_components=2, perplexity=5, max_iter=5000)
tsne_components = tsne.fit_transform(X_resampled)

# Creating a DataFrame with t-SNE components and the target variable
tsne_df = pd.DataFrame(data=tsne_components, columns=['TSNE 1', 'TSNE 2'])
tsne_df['Target'] = y_resampled

plt.figure(figsize=(8,6))
plt.scatter(tsne_df['TSNE 1'], tsne_df['TSNE 2'], c=tsne_df['Target'])
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')
plt.title('t-SNE of One-Hot Encoded Data')
plt.colorbar()
plt.show()


In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px

# Assuming X is your feature DataFrame and y is your target Series
tsne = TSNE(n_components=2, perplexity=100, max_iter=5000)
tsne_components = tsne.fit_transform(X_resampled)

# Creating a DataFrame with t-SNE components and the target variable
tsne_df = pd.DataFrame(data=tsne_components, columns=['TSNE 1', 'TSNE 2'])
tsne_df['Target'] = y_resampled

plt.figure(figsize=(8,6))
plt.scatter(tsne_df['TSNE 1'], tsne_df['TSNE 2'], c=tsne_df['Target'])
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')
plt.title('t-SNE of One-Hot Encoded Data')
plt.colorbar()
plt.show()


# Test using project data

In [ ]:
from utils.process_data import get_data
from models.preferences.data_utils import prepare_ml_data, get_child_data, initialize_children_data
from models.preferences.data_utils import plot_2d_mca, plot_3d_mca, plot_3d_mca_interactive, plot_scree_mca
from imblearn.over_sampling import SMOTE
child_data = get_child_data()
ingredient_df = get_data("data.csv")

preferences = initialize_children_data(child_data, ingredient_df)

# print(preferences)

X, y, df, label_encoder, preprocessor = prepare_ml_data(preferences, ingredient_df, child_data)


print(df.head())


# # Convert sparse matrix to dense format if necessary
X_dense = X.toarray() if hasattr(X, 'toarray') else X

# print(X_dense.shape)
# # Apply SMOTE
# smote = SMOTE(random_state=42)

# X_resampled, y_resampled = smote.fit_resample(X_dense, y)



In [ ]:
import pandas as pd
import prince

# Assuming df is your DataFrame with one-hot encoded categorical features
# Example data loading
# df = pd.read_csv('your_one_hot_encoded_data.csv')

# Apply MCA
mca = prince.MCA(n_components=2, n_iter=3, copy=True, check_input=True, engine='sklearn', random_state=42)
mca = mca.fit(df)

# Transform the data
df_mca = mca.transform(df)

# Print the explained variance
print(f"Explained Variance: {mca.explained_inertia_}")

# Plot the results
ax = mca.plot_coordinates(df, ax=None, figsize=(10, 8), show_row_points=True, show_column_points=True,
                          show_row_labels=False, show_column_labels=True)
ax.set_title('MCA of One-Hot Encoded Data')
ax.set_xlabel('MCA 1')
ax.set_ylabel('MCA 2')
plt.show()


# MCA

In [ ]:
from utils.process_data import get_data
from models.preferences.data_utils import prepare_ml_data, get_child_data, initialize_children_data
from models.preferences.data_utils import plot_2d_mca, plot_3d_mca, plot_3d_mca_interactive, plot_scree_mca
from imblearn.over_sampling import SMOTE
child_data = get_child_data()
ingredient_df = get_data("data.csv")
preferences = initialize_children_data(child_data, ingredient_df, seed=None)


X, y, df, label_encoder, preprocessor = prepare_ml_data(preferences, ingredient_df, child_data)

df.head()

import prince

mca = prince.MCA(
    n_components=3,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mca = mca.fit(df)


In [ ]:
import pandas as pd
import prince
import matplotlib.pyplot as plt
import seaborn as sns
# Separate the features and class labels
features = df.drop(columns=['preference'])
class_labels = df['preference']


# Apply MCA
mca = prince.MCA(n_components=2, n_iter=30, copy=True, check_input=True, engine='sklearn', random_state=42)
mca = mca.fit(features)

# Transform the data
df_mca = mca.transform(features)
df_mca['class_label'] = class_labels.values

# Get distinct colors for each class label
unique_labels = class_labels.unique()
palette = sns.color_palette("hsv", len(unique_labels))
color_dict = {label: palette[i] for i, label in enumerate(unique_labels)}

# Assign colors to each point based on the class label
colors = [color_dict[label] for label in df_mca['class_label']]

# Plot the results
fig, ax = plt.subplots(figsize=(10, 8))

# Scatter plot of MCA results, color-coded by class labels
scatter = ax.scatter(df_mca[0], df_mca[1], c=colors, alpha=0.6)

# Add a legend
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_dict[label], markersize=10, label=label) for label in unique_labels]
ax.legend(title='Class Label', handles=handles)

# Plot settings
ax.set_title('MCA of One-Hot Encoded Data')
ax.set_xlabel('MCA 1')
ax.set_ylabel('MCA 2')

plt.show()

In [ ]:
mca.eigenvalues_summary

In [ ]:
mca.plot(
    df,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=False
)


In [ ]:
mca.column_contributions_.head().style.format('{:.010%}')